In [43]:
import api_IBGE as ibge
from api_basedosdados import coleta_vinculos_rais
import pandas as pd
import basedosdados as bd

## Dados da PIA

In [ ]:
#checa metadados da PIA, sem especificar periodicidade não tras todos os dados
ibge.consulta_metadados(tabela='2221')

In [ ]:
df = ibge.consulta_ibge(tabela='2221',
                        periodos='1966|1967|1968|1969|1973|1974|1976|1977|1978|1979|1981|1982|1983|1984|1988|1989|1990|1992|1993|1994|1995',
                   variaveis='1150|1151|215|811', 
                   classificacao_categorias='12123[101229,101230]',
                   localidades='N1[all]')

df = ibge.limpa_resultado(df)

In [ ]:
df.tail(50)

In [ ]:
df.to_csv('pesquisa_industrial_anual.csv')

### Ajuste das moedas

In [ ]:
import pandas as pd
moedas_brasil = {
    (1500, 1833): "Real português (Réis)",
    (1833, 1942): "Real brasileiro (Réis)",
    (1942, 1967): "Cruzeiro (Cr$)",
    (1967, 1986): "Cruzeiro Novo (NCr$)",
    (1986, 1989): "Cruzado (Cz$)",
    (1989, 1990): "Cruzado Novo (NCz$)",
    (1990, 1993): "Cruzeiro (Cr$)",
    (1993, 1994): "Cruzeiro Real (CR$)",
    (1994, 2025): "Real (R$)"
}


def obter_moeda(ano):
    for intervalo, nome in moedas_brasil.items():
        if intervalo[0] <= ano < intervalo[1]:
            return nome
    return "Moeda desconhecida"

# Aplicar a conversão ao DataFrame
df.loc[42:,'Unidade'] = df.loc[42:,'ano'].astype(int).apply(obter_moeda)



In [ ]:
df.tail(50)

In [ ]:
df.to_csv('Dados/pesquisa_industrial_anual_1966_1995.csv')

## Dados Contas Nacionais

### Dados pós-1989
fonte -> https://seculoxx.ibge.gov.br/economicas/contas-nacionais


### Dados pré-1989
fonte ->https://seculoxx.ibge.gov.br/economicas/contas-nacionais

## Dados vinculos SF e ABC

In [12]:
municipios = [
    "3547809", "3548708", "3548807", "3513801", "3529401", "3543303", "3544103",
    "3300308", "3300407", "3301801", "3302809", "3303955", "3304003", "3543907",
    "3306305", "3302254", "3304110", "3304128", "3304201"
]

municipios_str = "('"+ "','".join(municipios) +"')"

In [32]:
cnae_1 = ['25119','31429','29530','29521','34100',
          '34207','34312','34320','34410','34428',
          '34436','34444','31607','34495','34509','29726']
cnae_2 = ['22111','27228','28542','29107','29204',
          '29301','29417','29425','29433','29441',
          '29450','29492','29492','29506','30504']

In [51]:
df, nome = coleta_vinculos_rais(codigo_municipio=municipios_str, ano_inicio=1995, billing_project_id="acessocaged", nome_arquivo='teste')

c:\Users\leoau\anaconda3\envs\bd_desindustrializacao\Lib\site-packages\pandas_gbq\gbq.py:320: LargeResultsWarning: Recommendation: Your results are 1.1 GiB. Consider using BigQuery DataFrames (https://bit.ly/bigframes-intro)to process large results with pandas compatible APIs with transparent SQL pushdown to BigQuery engine. This provides an opportunity to save on costs and improve performance. Please reach out to bigframes-feedback@google.com with any questions or concerns. To disable this message, run warnings.simplefilter('ignore', category=pandas_gbq.exceptions.LargeResultsWarning)
  final_df = connector.run_query(


Downloading: 100%|██████████|


In [52]:
df.to_csv('vinculos_sf_abc.csv')

### Dados de vinculos Industria Automotiva

In [50]:
cnae_1 = '''('25119','31429','29530','29521','34100',
          '34207','34312','34320','34410','34428',
          '34436','34444','31607','34495','34509','29726')'''
cnae_2 = '''('22111','27228','28542','29107','29204',
          '29301','29417','29425','29433','29441',
          '29450','29492','29492','29506','30504')'''

query = '''
            SELECT ano, sigla_uf, id_municipio, vinculo_ativo_3112,
            cnae_2, cnae_1,
            valor_remuneracao_media, valor_remuneracao_dezembro,
            valor_remuneracao_media_sm, valor_remuneracao_dezembro_sm,
            faixa_horas_contratadas,
            idade, sexo, raca_cor,
            grau_instrucao_1985_2005, grau_instrucao_apos_2005,  
            FROM  basedosdados.br_me_rais.microdados_vinculos
            WHERE ano >= 1995 AND 
            (cnae_2 IN {} OR cnae_1 IN {})
            AND vinculo_ativo_3112 = '1'
            '''.format(cnae_2, cnae_1)

df_todos = bd.read_sql(query,billing_project_id="acessocaged")
df_todos.to_csv('vinculos_automotiva.csv')

c:\Users\leoau\anaconda3\envs\bd_desindustrializacao\Lib\site-packages\pandas_gbq\gbq.py:320: LargeResultsWarning: Recommendation: Your results are 1.0 GiB. Consider using BigQuery DataFrames (https://bit.ly/bigframes-intro)to process large results with pandas compatible APIs with transparent SQL pushdown to BigQuery engine. This provides an opportunity to save on costs and improve performance. Please reach out to bigframes-feedback@google.com with any questions or concerns. To disable this message, run warnings.simplefilter('ignore', category=pandas_gbq.exceptions.LargeResultsWarning)
  final_df = connector.run_query(


Downloading: 100%|██████████|
